<a href="https://colab.research.google.com/github/sipocz/malavi/blob/main/Malavi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm Train.csv
fname="https://github.com/sipocz/malavi/raw/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv"
!wget https://github.com/sipocz/malavi/raw/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv

--2021-03-30 18:29:59--  https://github.com/sipocz/malavi/raw/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/malavi/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv [following]
--2021-03-30 18:30:00--  https://raw.githubusercontent.com/sipocz/malavi/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3158546 (3.0M) [text/plain]
Saving to: ‘Train.csv’

Train.csv           100%[===================>]   3.01M  --.-KB/s    in 0.1s    

2021-03-30 18:30:01 (28.3 MB/s) - ‘Train.csv’ saved [315854

In [ ]:
!head Train.csv

ID,Text,Label
ID_AASHwXxg," Mwangonde: Khansala wachinyamata Akamati achinyamata ndi atsogoleri a mawa, ambiri amaganiza kuti izi ndi nkhambakamwa chabe. Koma achinyamata ena, monga Lusubilo Mwangonde, akukwaniritsa akupherezetsa mawuwa osati pongolota kuti adzakhala, koma kutsogolera kumene chifukwa nthawi yawo yakwana. DAILES BANDA adacheza ndi Mwangonde, khansala wachinyama, yemwe akuimira Jumbo Ward mumzinda wa Mzuzu, motere:  Chisale watuluka nkumangidwanso  Sipakala waimitsa Nyumba ya Malamulo  Pa Wenela pasintha zedi Ali ndi masomphenya: Mwangonde Tikudziweni  Ndine Lusubilo Mwangonde, ndili ndi zaka 27 zakubadwa. Ndinabadwa mbanja la ana asanu ndipo ndine wachinayi kubadwa. Ndimachokera mmudzi mwa Mwamalopa, kwa Paramount Chief Kyungu mboma la Karonga. Sindili pabanja pakadalipano.
 Mbiri ya maphunziro anu ndi yotani? Maphunziro anga a pulaimale ndidachitira kusukula yapulaiveti ya Viphya mumzinda wa Mzuzu ndipo asekondale ndidachitira pa Phwezi Boys mboma la Rumphi. Ndili ndi 

In [ ]:
import pandas as pd
df=pd.read_csv("Train.csv",sep=",")

In [ ]:
len(df)

1436

In [30]:
labels=list(set(df.Label))

In [31]:
labels

['POLITICS',
 'ARTS AND CRAFTS',
 'LAW/ORDER',
 'HEALTH',
 'SOCIAL ISSUES',
 'SOCIAL',
 'SPORTS',
 'MUSIC',
 'FARMING',
 'WILDLIFE/ENVIRONMENT',
 'RELATIONSHIPS',
 'EDUCATION',
 'FLOODING',
 'OPINION/ESSAY',
 'ECONOMY',
 'TRANSPORT',
 'LOCALCHIEFS',
 'CULTURE',
 'WITCHCRAFT',
 'RELIGION']

In [32]:
label_index=[]
for i in list(df.Label):
    label_index.append(labels.index(i))

In [34]:
df["label_index"]=label_index

In [35]:
df.head()

,ID,Text,Label,label_index
0,ID_AASHwXxg,Mwangonde: Khansala wachinyamata Akamati achi...,POLITICS,0
1,ID_AGoFySzn,MCP siidakhutire ndi kalembera Chipani cha Ma...,POLITICS,0
2,ID_AGrrkBGP,Bungwe la MANEPO Lapempha Boma Liganizire Anth...,HEALTH,3
3,ID_AIJeigeG,Ndale zogawanitsa miyambo zanyanya Si zachile...,POLITICS,0
4,ID_APMprMbV,Nanga wapolisi ataphofomoka? Masiku ano sichi...,LAW/ORDER,2


In [ ]:
s1=set()
for i in range(len(df)):
    s1=s1.union(set(df.Text[i].replace(",","").replace(".","").upper().split()))
words=list(s1)

In [ ]:
words.sort()

In [ ]:
def findA():
    for inx,i in enumerate(words):
        if i=="A":
            print(inx)
            return(inx)
words=words[findA():]

1353


In [ ]:
lines=[]
for i in range(len(df)):
    s1=set(df.Text[i].replace(",","").replace(".","").upper().split())
    line=[]
    for w in s1:
        if w in words:
            line.append(words.index(w))
    lines.append(line)

In [ ]:
len(lines)

1436

In [16]:
ritka=[]

for line in lines:
    ritkaline=[0 for i in range(len(words))]
    for pos in line:
        ritkaline[pos]=1
    ritka.append(ritkaline)

In [19]:
!pip install tensorflow-addons

     |████████████████████████████████| 706kB 8.5MB/s 


In [20]:
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta,Adagrad,Nadam
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld
import tensorflow_addons as tfa


In [39]:
X=ritka
y=list(df.label_index)

In [42]:
clear_session()

kernel_reg_1=tf.keras.regularizers.L2(0.1)

input_size=len(X[0])
output_size=20
drop_frac0=0.7  #0.6 jó
drop_frac1=0.6  
'''
desc=[
        {"node":100,"activation":"swish","kernel_init":"lecun_normal"},
        {"node":120,"activation":"swish","kernel_init":"lecun_normal"},
        {"node":110,"activation":"swish","kernel_init":"lecun_normal"},
]
80X32 0.060
'''
desc=[
        {"node":200,"activation":"tanh","kernel_init":"lecun_normal"},
        {"node":220,"activation":"tanh","kernel_init":"lecun_normal"},
        {"node":210,"activation":"tanh","kernel_init":"lecun_normal"},
]

'''

desc=[
        {"node":200,"activation":"tanh","kernel_init":"lecun_normal"},
        {"node":220,"activation":"tanh","kernel_init":"lecun_normal"},
        {"node":210,"activation":"tanh","kernel_init":"lecun_normal"},
]
2x80x48: 0.0028 loss: msle learning rate=0.00005
'''

desc=[
        {"node":250,"activation":"tanh","kernel_init":"lecun_normal"},
        {"node":50,"activation":"tanh","kernel_init":"lecun_normal"},
        {"node":250,"activation":"tanh","kernel_init":"lecun_normal"},
       
      
       
      
]



input1=Input(shape=(input_size,))
layer_input=input1
for idx,_ in enumerate(desc):
    l1_out=Dense(desc[idx]["node"],activation=desc[idx]["activation"],kernel_initializer=desc[idx]["kernel_init"])(layer_input) # 202102262002 l1_out=Dense(370,activation="sigmoid")(input1)
    l2_out=Dropout(drop_frac0)(l1_out)
    layer_input=l2_out
    # 2600 as értékkel 0.033

#l3_out=Dense(2000,activation="sigmoid",kernel_initializer='lecun_normal',)(l2_out)
#l4_out=Dropout(drop_frac1)(l3_out)
#2000-vel jól megy


pred=Dense(output_size,)(l2_out)

model = Model(inputs=input1, outputs=pred)
optimizer=Adamax(learning_rate=0.00005,) #202102261859 ~0,0038: Adamax(learning_rate=0.01,) #SGD(lr=0.005, decay=1e-4, momentum=0.9)
optimizer=Nadam(learning_rate=0.00005,)
optimizer=tfa.optimizers.LazyAdam(0.001)
opt = tf.keras.optimizers.Adamax(learning_rate=0.00005)
optimizer = tfa.optimizers.Lookahead(opt)

model.compile(loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=["softmax"])

In [ ]:
history = model.fit( X,y, 
                        epochs=10, 
                        batch_size=3,validation_split=0.1,
                        
                        verbose=1
                              
                              )